In [1]:
import os, sys
sys.path.append('../../src')

from datetime import datetime
import pandas as pd

import visualization.visualize as vis
from data.load_dataset import load_dataset
from models.train_model import get_dataloaders, set_and_train_model
from models.model import get_model, evaluate_model, save_model

## Running all databases and rounds

In [2]:
num_epochs = 5
base_path='../../models/local_models/'

In [3]:
exec_time = datetime.now()
exec_id = f'{exec_time.year}-{exec_time.month}-{exec_time.day}_{exec_time.hour}h{exec_time.minute}m{exec_time.second}s'

training_stats = []
for round_number in [1,2,3,4,5]:
    for db_number in [1,2,3,4,5]:

        print(f'\n\nTraining round_number {round_number} db_number {db_number}:')
        model_id = f'db{db_number}_round{round_number}'
        model_id_old = f'db{db_number}_round{round_number-1}'
        model_name = f'{exec_id}_{model_id}'
        model_name_old = f'{exec_id}_{model_id_old}'
        
        df = load_dataset(db_number)
        dataloaders = get_dataloaders(df.sample(1000), round_number)

        ### Visualize a few images
        # Let's visualize a few training images so as to understand the data augmentations.
        # vis.plot_samples(dataloaders['train'], title=f'db_number {db_number} round_number {round_number} samples')        
        
        # Train the model
        model = get_model(model_name=model_name_old, base_path=base_path)
        model, model_hist = set_and_train_model(model, dataloaders, num_epochs=num_epochs)
        # vis.plot_model_hist(model_hist, f'Baseline Model db_number: {db_number} round_number: {round_number}')
        
        # Evaluate model predictions
        model_stats = evaluate_model(model, dataloaders, split='test')
        model_stats['round_number'] = round_number; model_stats['db_number'] = db_number
        training_stats.append(model_stats)
        # Saving current model:
        save_model(model, model_name, base_path=base_path)



Training round_number 1 db_number 1:
train_set size: 245
valid_set size: 73
test_set size: 73
total: 391

File Not Found: 2022-10-4_22h50m35s_db1_round0. Model will start with default initialization.
Cuda available: True. Model sento to device: cuda:0.

Epoch 0/4
----------
train Loss: 0.6878 Acc: 0.5551
valid Loss: 0.6604 Acc: 0.6301
0m 15s/epoch

Epoch 1/4
----------
train Loss: 0.6186 Acc: 0.6490
valid Loss: 0.6318 Acc: 0.6438
0m 12s/epoch

Epoch 2/4
----------
train Loss: 0.5719 Acc: 0.7102
valid Loss: 0.7556 Acc: 0.5479
0m 9s/epoch

Epoch 3/4
----------
train Loss: 0.5425 Acc: 0.7306
valid Loss: 0.6451 Acc: 0.6164
0m 8s/epoch

Epoch 4/4
----------
train Loss: 0.4181 Acc: 0.8163
valid Loss: 0.5837 Acc: 0.6575
0m 10s/epoch
Training complete in 0m 55s
Best val Acc: 0.657534

Classification report for test set:
              precision    recall  f1-score   support

         0.0       0.67      0.74      0.70        46
         1.0       0.45      0.37      0.41        27

    accura

In [4]:
# Saving statistics report:
training_stats = pd.DataFrame(training_stats)
training_stats.to_csv(f'{base_path}{exec_id}_training_stats.csv', index=False)

training_stats

,accuracy_score,balanced_accuracy_score,recall_score,precision_score,f1_score,confusion_matrix,round_number,db_number
0,0.602740,0.554750,0.370370,0.454545,0.408163,"[[34, 12], [17, 10]]",1,1
1,0.649351,0.585554,0.258065,0.666667,0.372093,"[[42, 4], [23, 8]]",1,2
2,0.670732,0.487121,0.090909,0.222222,0.129032,"[[53, 7], [20, 2]]",1,3
3,0.531646,0.519305,0.324324,0.500000,0.393443,"[[30, 12], [25, 12]]",1,4
4,0.584416,0.536466,0.290323,0.473684,0.360000,"[[36, 10], [22, 9]]",1,5
5,0.663636,0.655866,0.549020,0.666667,0.602151,"[[45, 14], [23, 28]]",2,1
6,0.663551,0.654839,0.600000,0.600000,0.600000,"[[44, 18], [18, 27]]",2,2
7,0.732759,0.723039,0.666667,0.680851,0.673684,"[[53, 15], [16, 32]]",2,3
8,0.616071,0.624194,0.700000,0.555556,0.619469,"[[34, 28], [15, 35]]",2,4
9,0.618182,0.618056,0.611111,0.611111,0.611111,"[[35, 21], [21, 33]]",2,5


In [6]:
training_stats = pd.read_csv(f'{base_path}{exec_id}_training_stats.csv')
training_stats

,accuracy_score,balanced_accuracy_score,recall_score,precision_score,f1_score,confusion_matrix,round_number,db_number
0,0.602740,0.554750,0.370370,0.454545,0.408163,[[34 12]\n [17 10]],1,1
1,0.649351,0.585554,0.258065,0.666667,0.372093,[[42 4]\n [23 8]],1,2
2,0.670732,0.487121,0.090909,0.222222,0.129032,[[53 7]\n [20 2]],1,3
3,0.531646,0.519305,0.324324,0.500000,0.393443,[[30 12]\n [25 12]],1,4
4,0.584416,0.536466,0.290323,0.473684,0.360000,[[36 10]\n [22 9]],1,5
5,0.663636,0.655866,0.549020,0.666667,0.602151,[[45 14]\n [23 28]],2,1
6,0.663551,0.654839,0.600000,0.600000,0.600000,[[44 18]\n [18 27]],2,2
7,0.732759,0.723039,0.666667,0.680851,0.673684,[[53 15]\n [16 32]],2,3
8,0.616071,0.624194,0.700000,0.555556,0.619469,[[34 28]\n [15 35]],2,4
9,0.618182,0.618056,0.611111,0.611111,0.611111,[[35 21]\n [21 33]],2,5
